In [9]:
import pandas as pd
import numpy as np
import pickle
from os import listdir

In [10]:
df_files = [f for f in listdir() if f[-2:] == '.p'] # list all file names of pickled dataframes

In [11]:
bus_line = 203 # bus line number of interest

In [12]:
journey_cols = [
    'timestamp', 
    'event', 
    'vehicle_id', 
    'line', 
    'longitude', 
    'latitude', 
    'direction', 
    'speed', 
    'station',
    'journey_number',
    'segment_number'
]

In [41]:
df2 = pd.DataFrame(columns=journey_cols).astype(dtype={
    'timestamp': 'object', 
    'event': 'object',
    'vehicle_id': 'int64',
    'line': 'int64',
    'longitude': 'float64', 
    'latitude': 'float64',
    'direction': 'float64',
    'speed': 'float64',
    'station': 'object',
    'journey_number': 'int64',
    'segment_number': 'int64'
})

In [42]:
bus_line = 203

In [44]:
journey_number = 1
# iterate over all pickled dataframes
for file in df_files:
    print(f'** Starting on new file: {file} **')
    df = pickle.load(open(file, 'rb'))
    # get the unique vehicle ids that drove bus line of interest
    vehicle_ids = df[(df['line'] == bus_line)]['vehicle_id'].unique()
    # iterate over all vehicle ids
    for vid in vehicle_ids:
        print(f'Startng on new vehicle with id: {vid}')
        vals = list()
        started = False
        # extract slice of df for this vehicle id and sort by timestamp
        df_ = df[df['vehicle_id'] == vid].copy().sort_values('timestamp')
        for row in df_.itertuples():
            # start collecting data at when JourneyStartedEvent fires
            # if the started event was fired at the starting position of interest
            # (longitude difference is less than 0.001)
            if not started \
            and row[2] == 'JourneyStartedEvent' \
            and row[4] == bus_line \
            and abs(row[5] - 58.414223) < 0.001:
                started = True
                segment_number = 1
                vals.append([x for x in row[1:]] + [journey_number, segment_number])
                continue
            
            # as long as the bus line is the same, we are on the same route, 
            # since we are only looking at a single vehicle and events are sorted by timestamp
            # keep collecting data until we hit a the Parkgatan station, where we write to dataframe
            # and start over
            if started and (row[4] == bus_line or row[4] == 0):
                # if we hit a JourneyStartedEvent or JourneyCompletedEvent after our initial
                # JourneyStartedEvent, we have passed Parkgatan station without firing an EnteredEvent
                # in that case, scrap this data and start over
                if row[2] in {'JourneyStartedEvent', 'JourneyCompletedEvent'}:
                    started = False
                    vals = list()
                    continue
                # if all is good, keep adding rows to the list for this journey
                vals.append([x for x in row[1:]] + [journey_number, segment_number])
                # if we hit EnteredEvent, increment segment_number if the station is not Parkgatan,
                # otherwise, if it is Parkgatan, we have reached the end for this journey
                if row[2] == 'EnteredEvent':
                    if row[9] == 'Parkgatan Link\\xf6ping':
                        df2 = df2.append(pd.DataFrame(data=vals, columns=journey_cols)).astype(dtype={
                            'timestamp': 'object', 
                            'event': 'object',
                            'vehicle_id': 'int64',
                            'line': 'int64',
                            'longitude': 'float64', 
                            'latitude': 'float64',
                            'direction': 'float64',
                            'speed': 'float64',
                            'station': 'object',
                            'journey_number': 'int64',
                            'segment_number': 'int64'
                        })
                        print(f'Successfully collected {journey_number} journeys!')
                        started = False
                        vals = list()
                        journey_number += 1
                    # since sometimes EnteredEvent fires right after the JourneyStartedEvent,
                    # we don't want to increment segment number if we are still at the starting station,
                    # (Rydsvägens ändhållplats)
                    elif row[9] != 'Rydsv\\xe4gens \\xe4ndh\\xe5llpl.':
                        segment_number += 1
                        
            # if bus line changed anywhere in the middle of this sequence, something went wrong
            # we then scrap the collected data and start over
            else:
                started = False
                vals = list()

** Starting on new file: 20180216-9.p **
Startng on new vehicle with id: 5479
Startng on new vehicle with id: 5484
Startng on new vehicle with id: 5422
Startng on new vehicle with id: 5490
Successfully collected 1 journeys!
Startng on new vehicle with id: 5441
Successfully collected 2 journeys!
Startng on new vehicle with id: 5431
Successfully collected 3 journeys!
Startng on new vehicle with id: 5478
Startng on new vehicle with id: 5457
Successfully collected 4 journeys!
Startng on new vehicle with id: 5485
Startng on new vehicle with id: 5418
Startng on new vehicle with id: 5428
Successfully collected 5 journeys!
** Starting on new file: 20180216-3.p **
Startng on new vehicle with id: 5436
Successfully collected 6 journeys!
Startng on new vehicle with id: 5453
Successfully collected 7 journeys!
Startng on new vehicle with id: 5481
Successfully collected 8 journeys!
Startng on new vehicle with id: 5415
Successfully collected 9 journeys!
Startng on new vehicle with id: 5501
Successfull

In [45]:
 pickle.dump(df2, open('buslines/bus203.p', 'wb'))

In [51]:
len(df2.index) * 90

5973570

In [49]:
for jn in df2['journey_number'].unique():
    print(len(df2[df2['journey_number'] == jn]['station'].unique()))

13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13


In [50]:
df2['line'].unique()

array([203,   0])

In [40]:
df2['station'].unique()

array([None, 'Rydsv\\xe4gens \\xe4ndh\\xe5llpl.',
       'Bj\\xf6rnk\\xe4rrsskolan', 'Als\\xe4ttersgatan',
       'Rydsv\\xe4gen 236', 'Rydsv\\xe4gen 168', 'M\\xe5rdtorpsgatan',
       'Ryd centrum', 'Ostbrickan', 'Solhaga', 'Gamla Link\\xf6ping',
       'Vallaplan', 'Parkgatan Link\\xf6ping'], dtype=object)

In [47]:
for seg in df2['journey_number'].unique():
    display(df2[(df2['journey_number'] == seg) & (df2['event'] != 'ObservedPositionEvent')])

,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,2018-02-16T17:56:00.0000000+01:00,JourneyStartedEvent,5490,203,58.414223,15.571064,-1.0,-1.0,None,1,1
1,2018-02-16T17:56:00.0000000+01:00,EnteredEvent,5490,203,58.414223,15.571064,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,1,1
3,2018-02-16T17:56:00.0000000+01:00,ArrivedEvent,5490,203,58.414223,15.571064,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,1,1
210,2018-02-16T17:59:27.0000000+01:00,DepartedEvent,5490,203,58.414196,15.571178,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,1,1
215,2018-02-16T17:59:30.0000000+01:00,ExitedEvent,5490,203,58.414196,15.571178,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,1,1
265,2018-02-16T18:00:20.0000000+01:00,EnteredEvent,5490,203,58.414570,15.565040,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,1,1
270,2018-02-16T18:00:24.0000000+01:00,ArrivedEvent,5490,203,58.414524,15.564896,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,1,2
299,2018-02-16T18:00:51.0000000+01:00,DepartedEvent,5490,203,58.414402,15.564636,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,1,2
307,2018-02-16T18:00:58.0000000+01:00,ExitedEvent,5490,203,58.414402,15.564636,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,1,2
341,2018-02-16T18:01:32.0000000+01:00,EnteredEvent,5490,203,58.413189,15.560198,-1.0,-1.0,Als\xe4ttersgatan,1,2


,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,2018-02-16T18:11:00.0000000+01:00,JourneyStartedEvent,5441,203,58.414196,15.571026,-1.0,-1.0,None,2,1
2,2018-02-16T18:11:00.0000000+01:00,ArrivedEvent,5441,203,58.414196,15.571026,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,2,1
3,2018-02-16T18:11:00.0000000+01:00,EnteredEvent,5441,203,58.414196,15.571026,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,2,1
100,2018-02-16T18:12:37.0000000+01:00,DepartedEvent,5441,203,58.414165,15.571195,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,2,1
107,2018-02-16T18:12:42.0000000+01:00,ExitedEvent,5441,203,58.414165,15.571195,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,2,1
169,2018-02-16T18:13:44.0000000+01:00,EnteredEvent,5441,203,58.414562,15.565059,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,2,1
176,2018-02-16T18:13:49.0000000+01:00,ArrivedEvent,5441,203,58.414539,15.564933,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,2,2
193,2018-02-16T18:14:05.0000000+01:00,DepartedEvent,5441,203,58.414417,15.564648,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,2,2
200,2018-02-16T18:14:11.0000000+01:00,ExitedEvent,5441,203,58.414417,15.564648,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,2,2
236,2018-02-16T18:14:47.0000000+01:00,EnteredEvent,5441,203,58.413189,15.560170,-1.0,-1.0,Als\xe4ttersgatan,2,2


,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,2018-02-16T17:41:00.0000000+01:00,JourneyStartedEvent,5431,203,58.414223,15.571052,-1.0,-1.0,None,3,1
2,2018-02-16T17:41:00.0000000+01:00,ArrivedEvent,5431,203,58.414223,15.571052,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,3,1
245,2018-02-16T17:45:02.0000000+01:00,DepartedEvent,5431,203,58.414173,15.571189,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,3,1
250,2018-02-16T17:45:06.0000000+01:00,ExitedEvent,5431,203,58.414173,15.571189,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,3,1
303,2018-02-16T17:45:59.0000000+01:00,EnteredEvent,5431,203,58.414566,15.565062,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,3,1
310,2018-02-16T17:46:04.0000000+01:00,ArrivedEvent,5431,203,58.414524,15.564887,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,3,2
317,2018-02-16T17:46:10.0000000+01:00,DepartedEvent,5431,203,58.414410,15.564644,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,3,2
322,2018-02-16T17:46:15.0000000+01:00,ExitedEvent,5431,203,58.414410,15.564644,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,3,2
357,2018-02-16T17:46:48.0000000+01:00,EnteredEvent,5431,203,58.413177,15.560134,-1.0,-1.0,Als\xe4ttersgatan,3,2
361,2018-02-16T17:46:52.0000000+01:00,ArrivedEvent,5431,203,58.413151,15.559995,-1.0,-1.0,Als\xe4ttersgatan,3,3


,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,2018-02-16T18:28:09.0000000+01:00,JourneyStartedEvent,5457,203,58.414230,15.571053,-1.0,-1.0,None,4,1
1,2018-02-16T18:28:09.0000000+01:00,EnteredEvent,5457,203,58.414230,15.571053,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,4,1
2,2018-02-16T18:28:09.0000000+01:00,ArrivedEvent,5457,203,58.414230,15.571053,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,4,1
76,2018-02-16T18:29:22.0000000+01:00,DepartedEvent,5457,203,58.414177,15.571209,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,4,1
81,2018-02-16T18:29:26.0000000+01:00,ExitedEvent,5457,203,58.414177,15.571209,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,4,1
131,2018-02-16T18:30:16.0000000+01:00,EnteredEvent,5457,203,58.414581,15.565084,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,4,1
137,2018-02-16T18:30:20.0000000+01:00,ArrivedEvent,5457,203,58.414528,15.564923,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,4,2
149,2018-02-16T18:30:31.0000000+01:00,DepartedEvent,5457,203,58.414417,15.564672,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,4,2
155,2018-02-16T18:30:36.0000000+01:00,ExitedEvent,5457,203,58.414417,15.564672,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,4,2
190,2018-02-16T18:31:11.0000000+01:00,EnteredEvent,5457,203,58.413193,15.560168,-1.0,-1.0,Als\xe4ttersgatan,4,2


,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,2018-02-16T19:18:00.0000000+01:00,JourneyStartedEvent,5428,203,58.414215,15.571060,-1.0,-1.0,None,5,1
2,2018-02-16T19:18:00.0000000+01:00,EnteredEvent,5428,203,58.414215,15.571060,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,5,1
3,2018-02-16T19:18:00.0000000+01:00,ArrivedEvent,5428,203,58.414215,15.571060,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,5,1
124,2018-02-16T19:20:00.0000000+01:00,DepartedEvent,5428,203,58.414185,15.571180,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,5,1
129,2018-02-16T19:20:04.0000000+01:00,ExitedEvent,5428,203,58.414185,15.571180,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,5,1
185,2018-02-16T19:20:59.0000000+01:00,EnteredEvent,5428,203,58.414608,15.565063,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,5,1
192,2018-02-16T19:21:05.0000000+01:00,ArrivedEvent,5428,203,58.414547,15.564878,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,5,2
224,2018-02-16T19:21:36.0000000+01:00,DepartedEvent,5428,203,58.414421,15.564623,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,5,2
231,2018-02-16T19:21:42.0000000+01:00,ExitedEvent,5428,203,58.414421,15.564623,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,5,2
261,2018-02-16T19:22:11.0000000+01:00,EnteredEvent,5428,203,58.413197,15.560114,-1.0,-1.0,Als\xe4ttersgatan,5,2


...

,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,2018-02-16T10:44:00.0000000+01:00,JourneyStartedEvent,5432,203,58.414230,15.571033,-1.0,-1.0,None,64,1
94,2018-02-16T10:46:41.0000000+01:00,DepartedEvent,5432,203,58.414162,15.571179,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,64,1
104,2018-02-16T10:46:51.0000000+01:00,ExitedEvent,5432,203,58.414162,15.571179,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,64,1
174,2018-02-16T10:48:00.0000000+01:00,EnteredEvent,5432,203,58.414574,15.565084,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,64,1
180,2018-02-16T10:48:05.0000000+01:00,ArrivedEvent,5432,203,58.414516,15.564918,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,64,2
201,2018-02-16T10:48:25.0000000+01:00,DepartedEvent,5432,203,58.414425,15.564671,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,64,2
208,2018-02-16T10:48:30.0000000+01:00,ExitedEvent,5432,203,58.414425,15.564671,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,64,2
241,2018-02-16T10:49:03.0000000+01:00,EnteredEvent,5432,203,58.413193,15.560129,-1.0,-1.0,Als\xe4ttersgatan,64,2
247,2018-02-16T10:49:08.0000000+01:00,ArrivedEvent,5432,203,58.413143,15.559994,-1.0,-1.0,Als\xe4ttersgatan,64,3
261,2018-02-16T10:49:20.0000000+01:00,DepartedEvent,5432,203,58.413052,15.559735,-1.0,-1.0,Als\xe4ttersgatan,64,3


,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,2018-02-16T10:54:00.0000000+01:00,JourneyStartedEvent,5428,203,58.414196,15.571057,-1.0,-1.0,None,65,1
145,2018-02-16T10:56:23.0000000+01:00,DepartedEvent,5428,203,58.414158,15.571171,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,65,1
149,2018-02-16T10:56:26.0000000+01:00,ExitedEvent,5428,203,58.414158,15.571171,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,65,1
203,2018-02-16T10:57:20.0000000+01:00,EnteredEvent,5428,203,58.414570,15.565069,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,65,1
210,2018-02-16T10:57:26.0000000+01:00,ArrivedEvent,5428,203,58.414509,15.564903,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,65,2
228,2018-02-16T10:57:43.0000000+01:00,DepartedEvent,5428,203,58.414398,15.564664,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,65,2
237,2018-02-16T10:57:50.0000000+01:00,ExitedEvent,5428,203,58.414398,15.564664,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,65,2
272,2018-02-16T10:58:24.0000000+01:00,EnteredEvent,5428,203,58.413185,15.560197,-1.0,-1.0,Als\xe4ttersgatan,65,2
279,2018-02-16T10:58:31.0000000+01:00,ArrivedEvent,5428,203,58.413128,15.559962,-1.0,-1.0,Als\xe4ttersgatan,65,3
293,2018-02-16T10:58:43.0000000+01:00,DepartedEvent,5428,203,58.413074,15.559704,-1.0,-1.0,Als\xe4ttersgatan,65,3


,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,2018-02-16T11:04:00.0000000+01:00,JourneyStartedEvent,5453,203,58.414223,15.571030,-1.0,-1.0,None,66,1
208,2018-02-16T11:07:26.0000000+01:00,DepartedEvent,5453,203,58.414165,15.571184,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,66,1
216,2018-02-16T11:07:33.0000000+01:00,ExitedEvent,5453,203,58.414165,15.571184,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,66,1
272,2018-02-16T11:08:29.0000000+01:00,EnteredEvent,5453,203,58.414577,15.565037,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,66,1
278,2018-02-16T11:08:34.0000000+01:00,ArrivedEvent,5453,203,58.414543,15.564932,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,66,2
288,2018-02-16T11:08:42.0000000+01:00,DepartedEvent,5453,203,58.414413,15.564668,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,66,2
295,2018-02-16T11:08:48.0000000+01:00,ExitedEvent,5453,203,58.414413,15.564668,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,66,2
328,2018-02-16T11:09:21.0000000+01:00,EnteredEvent,5453,203,58.413204,15.560146,-1.0,-1.0,Als\xe4ttersgatan,66,2
335,2018-02-16T11:09:27.0000000+01:00,ArrivedEvent,5453,203,58.413174,15.560001,-1.0,-1.0,Als\xe4ttersgatan,66,3
348,2018-02-16T11:09:39.0000000+01:00,DepartedEvent,5453,203,58.413078,15.559730,-1.0,-1.0,Als\xe4ttersgatan,66,3


,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,2018-02-16T11:14:00.0000000+01:00,JourneyStartedEvent,5489,203,58.414223,15.571052,-1.0,-1.0,None,67,1
131,2018-02-16T11:16:09.0000000+01:00,DepartedEvent,5489,203,58.414173,15.571190,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,67,1
137,2018-02-16T11:16:14.0000000+01:00,ExitedEvent,5489,203,58.414173,15.571190,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,67,1
186,2018-02-16T11:17:02.0000000+01:00,EnteredEvent,5489,203,58.414574,15.565019,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,67,1
193,2018-02-16T11:17:08.0000000+01:00,ArrivedEvent,5489,203,58.414524,15.564874,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,67,2
220,2018-02-16T11:17:34.0000000+01:00,DepartedEvent,5489,203,58.414413,15.564614,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,67,2
227,2018-02-16T11:17:40.0000000+01:00,ExitedEvent,5489,203,58.414413,15.564614,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,67,2
259,2018-02-16T11:18:11.0000000+01:00,EnteredEvent,5489,203,58.413197,15.560167,-1.0,-1.0,Als\xe4ttersgatan,67,2
265,2018-02-16T11:18:16.0000000+01:00,ArrivedEvent,5489,203,58.413151,15.559999,-1.0,-1.0,Als\xe4ttersgatan,67,3
277,2018-02-16T11:18:27.0000000+01:00,DepartedEvent,5489,203,58.413063,15.559733,-1.0,-1.0,Als\xe4ttersgatan,67,3


,timestamp,event,vehicle_id,line,longitude,latitude,direction,speed,station,journey_number,segment_number
0,2018-02-16T11:24:00.0000000+01:00,JourneyStartedEvent,5427,203,58.414249,15.571078,-1.0,-1.0,None,68,1
2,2018-02-16T11:24:00.0000000+01:00,EnteredEvent,5427,203,58.414249,15.571078,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,68,1
3,2018-02-16T11:24:00.0000000+01:00,ArrivedEvent,5427,203,58.414249,15.571078,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,68,1
133,2018-02-16T11:26:09.0000000+01:00,DepartedEvent,5427,203,58.414215,15.571172,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,68,1
136,2018-02-16T11:26:11.0000000+01:00,ExitedEvent,5427,203,58.414215,15.571172,-1.0,-1.0,Rydsv\xe4gens \xe4ndh\xe5llpl.,68,1
177,2018-02-16T11:26:52.0000000+01:00,EnteredEvent,5427,203,58.414589,15.565061,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,68,1
184,2018-02-16T11:26:57.0000000+01:00,ArrivedEvent,5427,203,58.414539,15.564903,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,68,2
198,2018-02-16T11:27:11.0000000+01:00,DepartedEvent,5427,203,58.414394,15.564607,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,68,2
205,2018-02-16T11:27:16.0000000+01:00,ExitedEvent,5427,203,58.414394,15.564607,-1.0,-1.0,Bj\xf6rnk\xe4rrsskolan,68,2
233,2018-02-16T11:27:44.0000000+01:00,EnteredEvent,5427,203,58.413189,15.560116,-1.0,-1.0,Als\xe4ttersgatan,68,2
